In [3]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv("../datasets_raw/db_nl_preprocessed-edit.csv")  # предположим, что файл называется data.csv
print("Размер набора данных:", df.shape)
print(df.columns.tolist())  # Названия столбцов

# 1) Обработка пропусков: заполним отсутствующие значения медианой по столбцу
df = df.fillna(df.median(numeric_only=True))

# Проверим, остались ли пропуски
print("Осталось пропусков:\n", df.isna().sum())

Размер набора данных: (1962, 47)
['возраст', 'пол', 'масса_тела', 'рост', 'холестерин', 'нас._жир', 'натрий_na', 'сахара', 'энергия', 'вода', 'пищ._волокна', 'мононенас._жир', 'белки', 'полиненас._жир', 'калий_k', 'крахмал', 'жиры', 'кальций_ca', 'углеводы', 'олеиновая_кислота', 'магний_mg', 'a', 'b1_тиамин', 'b2_рибофлав.', 'b5_пантотен._кис.', 'b6_пиридоксин', 'b9_фолаты', 'b12_кобаламин', 'b4_холин', 'фосфор_p', 'омега-3', 'железо_fe', 'омега-6', 'c', 'e_α-токоферол', 'фенилаланин', 'pp_ниацин', 'йод_i', 'медь_cu', 'селен_se', 'цинк_zn', 'профессия_работники_преимущественно_умственного_труда', 'профессия_работники_занятые_легким_физическим_трудом', 'спорт_легкий_спорт', 'спорт_не_занимаюсь', 'bmi', 'target']
Осталось пропусков:
 возраст                                                  0
пол                                                      0
масса_тела                                               0
рост                                                     0
холестерин            

In [4]:
# 2) Удаление сильно коррелированных признаков
corr_matrix = df.drop(columns='target').corr().abs()        # корреляции по модулю между признаками (без целевой)
upper = corr_matrix.where( np.triu(np.ones_like(corr_matrix), k=1).astype(bool) )
high_corr_cols = [col for col in upper.columns if any(upper[col] > 0.90)]
print("Признаки с r>0.90:", high_corr_cols)

# Удалим эти признаки из датасета
df_reduced = df.drop(columns=high_corr_cols)
print("Осталось признаков:", len(df_reduced.columns) - 1)  # вычитаем целевой столбец


Признаки с r>0.90: ['жиры', 'олеиновая_кислота', 'магний_mg', 'b2_рибофлав.', 'b6_пиридоксин', 'фосфор_p', 'железо_fe', 'омега-6', 'e_α-токоферол', 'фенилаланин', 'pp_ниацин', 'селен_se', 'bmi']
Осталось признаков: 33


In [5]:
from sklearn.model_selection import train_test_split

X = df_reduced.drop(columns='target').values
y = df_reduced['target'].values

# Разобьем на train, val, test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42)

print("Размер обучающей выборки:", X_train.shape[0])
print("Размер валидационной выборки:", X_val.shape[0])
print("Размер тестовой выборки:", X_test.shape[0])


Размер обучающей выборки: 1373
Размер валидационной выборки: 294
Размер тестовой выборки: 295


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

# Проверим: средние обучающих признаков ~0, СКО ~1
print("Среднее обуч. признаков:", X_train_scaled.mean(axis=0))
print("СКО обуч. признаков:", X_train_scaled.std(axis=0))


Среднее обуч. признаков: [-2.15575716e-16 -4.80314987e-17 -7.13680147e-16 -2.43715382e-16
  1.26844800e-15 -3.26456512e-15  8.06387408e-16 -9.62934515e-16
 -2.74830737e-15 -9.17672509e-16 -1.98586797e-15 -2.39922996e-15
 -1.40948999e-15  1.00930836e-15 -4.37741613e-16 -1.50401662e-15
 -5.85919595e-16  8.02182630e-16  1.28569163e-16  1.56789690e-16
 -4.23235130e-15  4.85732681e-16 -9.40899863e-16 -1.04440208e-15
 -2.51801493e-16  5.16184469e-18  4.96244625e-16  1.42319595e-15
 -6.35568316e-16 -7.95673311e-17 -9.88688778e-16  1.47005496e-16
  1.84848495e-16]
СКО обуч. признаков: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [11]:
import torch
from kan import KAN  # импорт KAN из библиотеки PyKAN

# Определим архитектуру: [число входов, число скрытых, число выходов]
n_features = X_train_scaled.shape[1]
model = KAN(width=[n_features, 10, 2], k=3, grid=5, seed=0)  
# width = [33, 10, 2]: 33 входа -> 10 скрытых -> 2 выхода; k=3 (кубический сплайн), grid=5 (узлов для сплайна)

# Подготовим данные для PyTorch
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_val_t   = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_t   = torch.tensor(y_val, dtype=torch.long)

# Обучение модели KAN
dataset = {
    'train_input': X_train_t, 'train_label': y_train_t,
    'test_input':  X_val_t,   'test_label':  y_val_t
}
# results = model.train(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())
results = model.fit(
    dataset,
    opt="LBFGS",
    steps=100,
    loss_fn=torch.nn.CrossEntropyLoss()
)

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 4.09e-04 | test_loss: 5.05e+00 | reg: 5.97e+02 | : 100%|█| 100/100 [00:09<00:00, 10.09

saving model version 0.1


In [12]:
# Оценка точности на обучающей и валидационной выборках

train_preds = torch.argmax(model(X_train_t), dim=1).numpy()
val_preds   = torch.argmax(model(X_val_t), dim=1).numpy()
from sklearn.metrics import accuracy_score, f1_score
print("Точность (Train):", accuracy_score(y_train, train_preds))
print("Точность (Val):", accuracy_score(y_val, val_preds))
print("F1-мера (Val):", f1_score(y_val, val_preds))


Точность (Train): 1.0
Точность (Val): 0.6904761904761905
F1-мера (Val): 0.6690909090909091


In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
rf_val_preds = rf.predict(X_val_scaled)
print("Точность Random Forest (Val):", accuracy_score(y_val, rf_val_preds))


Точность Random Forest (Val): 0.7619047619047619
